<a href="https://colab.research.google.com/github/ssreepathy/COGS108_Repo/blob/main/DataCheckpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# COGS 108 - Data Checkpoint
## Names:
- Frida Mancilla
- Gabriella Bulacan
- Tatsuya Aizawa
- Murad Ali Gondal
- Sarat Sreepathy

## Research Question
Given that the efficiency of a country’s healthcare system can be measured by both economic indicators (GDP Expenditure) and health indicators (childhood mortality, vaccination coverage, insurance rate), how much does the quality of healthcare in the U.S contribute to the change in life expectancy from 1980 to 2016?

## Dataset(s)

1.
- Dataset Name: Life Expectancy at Birth (World Bank data) (1960-2016)
- Link to the dataset: https://ourworldindata.org/life-expectancy
- Number of observations: 13,748
- Description: This dataset contains number of years of life expectancy at birth from multiple countries around the world (includes U.S.)

2.
- Dataset Name: U.S National Health Expenditure Summary (1960-2016)
- Link to the dataset: https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/NationalHealthExpendData/NationalHealthAccountsHistorical
- Number of observations: 34
- Description: Contains info on healthcare expenditure/consumption, GDP per capita, U.S population, etc.

3.
- Dataset Name: Childhood Mortality (1950-2017)
- Link to the dataset:
https://ourworldindata.org/child-mortality-in-the-past 
- Number of observations: 8,345
- Description: This dataset contains the mortality rates of children that died before reaching 15.

4.
- Dataset Name: Annual Number of Deaths (1950-2019) 
- Link to the dataset: https://ourworldindata.org/grapher/annual-number-of-deaths-by-world-region
- Number of observations: 17,220
-  Description: Contains annual number of deaths in the entire world (includes U.S)

5.
- Dataset Name: Vaccination Coverage by Income (1980-2017)
- Link to the dataset: https://ourworldindata.org/grapher/vaccination-coverage-by-income-in
- Number of observations: 47,407
- Description: Contains the percentage coverage of one-year olds that were vaccinated against diphtheria, pertussis, and tetanus.

6.
- Dataset name: Insurance Rate (1963-2015)
- Link to the dataset:
https://ourworldindata.org/grapher/percentage-of-persons-without-health-insurance-coverage-us 
- Number of observations: 39
- Description: Contains percentage of persons in the U.S without health insurance.


Since we have multiple dataset, we will join the datasets by year. The earliest year throughout all datasets is 1980 and latest is 2016. This will be our year window of a total of 36 years.

## Setup

In [1]:
import pandas as pd

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

## Data Cleaning

### 1. National Health Expenditure Summary (NHE)

In [3]:
nhe = pd.read_excel('NHE19_Summary.xlsx')

nhe.head()

,"Table 1 \nNational Health Expenditures; Aggregate and Per Capita Amounts, Annual Percent Change and Percent Distribution: Calendar Years 1960-2019",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60
0,Expenditure Amount,1960.0,1961.0,1962.0,1963.0,1964.0,1965.0,1966.0,1967.0,1968.0,1969.0,1970.0,1971.0,1972.0,1973.0,1974.0,1975.0,1976.0,1977.0,1978.0,1979.0,1980.0,1981.0,1982.0,1983.0,1984.0,1985.0,1986.0,1987.0,1988.0,1989.0,1990.0,1991.0,1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,1999.0,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0
1,National Health Expenditures (Amount in Billions),27.1,29.1,31.8,34.6,38.2,41.6,45.8,51.2,58.0,65.4,74.1,82.4,92.4,102.7,116.3,132.7,152.0,172.6,194.0,219.7,253.2,293.6,330.9,364.8,401.9,439.9,472.3,514.5,576.7,642.2,718.8,786.1,852.4,915.1,966.0,1020.6,1074.0,1133.4,1198.9,1273.6,1365.6,1482.9,1630.6,1769.8,1899.5,2029.5,2166.3,2305.1,2402.5,2492.8,2589.7,2676.4,2783.1,2859.5,3008.3,3177.7,3324.5,3465.9,3629.7,3795.4
2,Health Consumption Expenditures,24.6,26.2,28.3,30.8,33.9,36.9,40.8,46.0,52.2,58.4,66.3,73.6,82.5,92.2,104.9,119.9,138.0,158.3,178.0,202.2,232.7,270.1,304.2,335.5,371.0,408.7,440.0,478.8,536.8,598.4,670.2,734.0,795.3,854.2,902.4,955.1,1006.9,1061.8,1120.3,1189.2,1279.9,1393.0,1529.1,1661.3,1782.6,1904.0,2033.3,2158.9,2246.2,2345.6,2437.6,2517.7,2619.9,2696.4,2848.7,3013.7,3157.5,3284.4,3439.5,3593.7
3,Personal Health Care,23.1,24.6,26.5,29.0,31.8,34.4,38.0,43.0,48.6,54.9,62.4,68.7,76.4,85.3,97.8,112.1,128.1,145.1,162.4,184.9,214.3,248.6,279.5,308.0,337.9,372.6,405.4,444.4,495.1,547.0,611.9,672.7,728.4,775.7,817.3,866.8,915.0,966.1,1019.6,1079.2,1156.9,1257.0,1366.1,1476.5,1587.8,1697.6,1808.6,1922.3,2008.4,2106.5,2181.6,2254.9,2347.2,2409.1,2533.6,2686.2,2813.5,2928.3,3048.3,3207.0
4,Government Administration and Net...,1.1,1.2,1.3,1.3,1.5,1.8,2.0,2.1,2.6,2.4,2.6,3.2,4.3,4.8,4.5,4.9,6.7,9.3,11.1,11.9,11.9,13.9,16.1,18.3,23.3,24.9,22.2,20.8,26.5,33.7,38.3,39.2,42.5,51.7,55.4,57.2,59.5,60.9,63.3,69.3,79.9,89.2,110.8,130.6,139.8,149.2,163.8,170.7,166.3,164.9,180.3,188.4,195.5,205.9,230.8,241.8,255.6,264.1,296.8,288.9


In [4]:
# Transpose National Healthcare Expenditure df so that year is a variable.
nhe = nhe.transpose()

nhe.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33
"Table 1 \nNational Health Expenditures; Aggregate and Per Capita Amounts, Annual Percent Change and Percent Distribution: Calendar Years 1960-2019",Expenditure Amount,National Health Expenditures (Amount in Billions),Health Consumption Expenditures,Personal Health Care,Government Administration and Net...,Government Public Health Activities,Investment,U.S. Population1 (Millions),Gross Domestic Product2 (Amount in Billions),National Health Expenditures (Annual Percent C...,Health Consumption Expenditures,Personal Health Care,Government Administration and Net...,Government Public Health Activities,Investment,U.S. Population1,Gross Domestic Product2,National Health Expenditures (Percent Distribu...,Health Consumption Expenditures,Personal Health Care,Government Administration and Net...,Government Public Health Activities,Investment,National Health Expenditures (Per Capita Amount),Health Consumption Expenditures,Personal Health Care,Government Administration and Net...,Government Public Health Activities,Investment,National Health Expenditures as a Percent of G...,1U.S. Bureau of the Census. Census resident-b...,"2U.S. Department of Commerce, Bureau of Econom...",NOTE: Numbers and percents may not add to tot...,SOURCE: Centers for Medicare & Medicaid Servi...
Unnamed: 1,1960,27.1,24.6,23.1,1.1,0.4,2.6,186.2,542.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,90.5,85.3,3.9,1.4,9.5,146,132,124,6,2,14,5,NaN,NaN,NaN,NaN
Unnamed: 2,1961,29.1,26.2,24.6,1.2,0.4,2.9,189.2,562.2,7.1,6.6,6.5,8.8,10.3,11.9,1.6,3.7,100,90.1,84.7,4,1.4,9.9,154,138,130,6,2,15,5.2,NaN,NaN,NaN,NaN
Unnamed: 3,1962,31.8,28.3,26.5,1.3,0.5,3.5,192,603.9,9.3,7.9,7.7,10.8,11.5,21.8,1.5,7.4,100,89,83.5,4,1.4,11,165,147,138,7,2,18,5.3,NaN,NaN,NaN,NaN
Unnamed: 4,1963,34.6,30.8,29,1.3,0.5,3.8,194.8,637.5,8.8,9,9.2,3.1,11.6,7.3,1.4,5.6,100,89.1,83.8,3.8,1.5,10.9,177,158,149,7,3,19,5.4,NaN,NaN,NaN,NaN


In [5]:
# Column names are stored in first row of the df, we need to change this accordingly.
nhe.columns = nhe.iloc[0]
nhe = nhe.rename(columns={'Expenditure Amount': 'Year'})
nhe = nhe.iloc[1:]
nhe = nhe.rename_axis(None, axis=1) # (Drop index header.)

nhe.head()

,Year,National Health Expenditures (Amount in Billions),Health Consumption Expenditures,Personal Health Care,Government Administration and Net Cost of Health Insurance,Government Public Health Activities,Investment,U.S. Population1 (Millions),Gross Domestic Product2 (Amount in Billions),National Health Expenditures (Annual Percent Change),Health Consumption Expenditures,Personal Health Care,Government Administration and Net Cost of Health Insurance,Government Public Health Activities,Investment,U.S. Population1,Gross Domestic Product2,National Health Expenditures (Percent Distribution),Health Consumption Expenditures,Personal Health Care,Government Administration and Net Cost of Health Insurance,Government Public Health Activities,Investment,National Health Expenditures (Per Capita Amount),Health Consumption Expenditures,Personal Health Care,Government Administration and Net Cost of Health Insurance,Government Public Health Activities,Investment,National Health Expenditures as a Percent of Gross Domestic Product (Percent),1U.S. Bureau of the Census. Census resident-based population less armed forces overseas and population of outlying areas.,"2U.S. Department of Commerce, Bureau of Economic Analysis.",NOTE: Numbers and percents may not add to totals because of rounding. Dollar amounts shown are in current dollars.,"SOURCE: Centers for Medicare & Medicaid Services, Office of the Actuary, National Health Statistics Group; U.S. Department of Commerce, Bureau of Economic Analysis; and U.S. Bureau of the Census."
Unnamed: 1,1960,27.1,24.6,23.1,1.1,0.4,2.6,186.2,542.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,90.5,85.3,3.9,1.4,9.5,146,132,124,6,2,14,5,NaN,NaN,NaN,NaN
Unnamed: 2,1961,29.1,26.2,24.6,1.2,0.4,2.9,189.2,562.2,7.1,6.6,6.5,8.8,10.3,11.9,1.6,3.7,100,90.1,84.7,4,1.4,9.9,154,138,130,6,2,15,5.2,NaN,NaN,NaN,NaN
Unnamed: 3,1962,31.8,28.3,26.5,1.3,0.5,3.5,192,603.9,9.3,7.9,7.7,10.8,11.5,21.8,1.5,7.4,100,89,83.5,4,1.4,11,165,147,138,7,2,18,5.3,NaN,NaN,NaN,NaN
Unnamed: 4,1963,34.6,30.8,29,1.3,0.5,3.8,194.8,637.5,8.8,9,9.2,3.1,11.6,7.3,1.4,5.6,100,89.1,83.8,3.8,1.5,10.9,177,158,149,7,3,19,5.4,NaN,NaN,NaN,NaN
Unnamed: 5,1964,38.2,33.9,31.8,1.5,0.6,4.3,197.4,684.5,10.7,10.1,9.8,14,12.5,15.7,1.4,7.4,100,88.6,83.2,3.9,1.5,11.4,194,172,161,8,3,22,5.6,NaN,NaN,NaN,NaN


In [6]:
# Last 4 columns we orginally rows that contained info on the dataset (footnotes), we can remove these as well.
nhe = nhe.iloc[:,:-4]
nhe = nhe.reset_index(drop=True)

nhe.head()

,Year,National Health Expenditures (Amount in Billions),Health Consumption Expenditures,Personal Health Care,Government Administration and Net Cost of Health Insurance,Government Public Health Activities,Investment,U.S. Population1 (Millions),Gross Domestic Product2 (Amount in Billions),National Health Expenditures (Annual Percent Change),Health Consumption Expenditures,Personal Health Care,Government Administration and Net Cost of Health Insurance,Government Public Health Activities,Investment,U.S. Population1,Gross Domestic Product2,National Health Expenditures (Percent Distribution),Health Consumption Expenditures,Personal Health Care,Government Administration and Net Cost of Health Insurance,Government Public Health Activities,Investment,National Health Expenditures (Per Capita Amount),Health Consumption Expenditures,Personal Health Care,Government Administration and Net Cost of Health Insurance,Government Public Health Activities,Investment,National Health Expenditures as a Percent of Gross Domestic Product (Percent)
0,1960,27.1,24.6,23.1,1.1,0.4,2.6,186.2,542.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,90.5,85.3,3.9,1.4,9.5,146,132,124,6,2,14,5
1,1961,29.1,26.2,24.6,1.2,0.4,2.9,189.2,562.2,7.1,6.6,6.5,8.8,10.3,11.9,1.6,3.7,100,90.1,84.7,4,1.4,9.9,154,138,130,6,2,15,5.2
2,1962,31.8,28.3,26.5,1.3,0.5,3.5,192,603.9,9.3,7.9,7.7,10.8,11.5,21.8,1.5,7.4,100,89,83.5,4,1.4,11,165,147,138,7,2,18,5.3
3,1963,34.6,30.8,29,1.3,0.5,3.8,194.8,637.5,8.8,9,9.2,3.1,11.6,7.3,1.4,5.6,100,89.1,83.8,3.8,1.5,10.9,177,158,149,7,3,19,5.4
4,1964,38.2,33.9,31.8,1.5,0.6,4.3,197.4,684.5,10.7,10.1,9.8,14,12.5,15.7,1.4,7.4,100,88.6,83.2,3.9,1.5,11.4,194,172,161,8,3,22,5.6


### 2. Life Expentancy data (World Bank)

In [7]:
life_expectancy = pd.read_csv('life-expectancy-at-birth-total-years.csv')

life_expectancy.head()

,Entity,Code,Year,"Life expectancy at birth, total (years)"
0,Afghanistan,AFG,1960,32.292
1,Afghanistan,AFG,1961,32.742
2,Afghanistan,AFG,1962,33.185
3,Afghanistan,AFG,1963,33.624
4,Afghanistan,AFG,1964,34.060


In [8]:
# We only need to grab colummns containing info on the US and >= 1980
life_expectancy = life_expectancy[life_expectancy['Entity'] == 'United States']
life_expectancy = life_expectancy.drop(columns=['Entity', 'Code'])
life_expectancy = life_expectancy[life_expectancy['Year'] >= 1980].reset_index(drop=True)
life_expectancy = life_expectancy.reset_index(drop=True)

life_expectancy.head()


,Year,"Life expectancy at birth, total (years)"
0,1980,73.609756
1,1981,74.009756
2,1982,74.360976
3,1983,74.463415
4,1984,74.563415


In [9]:
# Starting to merge them by year into a single dataframe, datum.

datum = pd.merge(nhe, life_expectancy, on='Year')
datum.head()

,Year,National Health Expenditures (Amount in Billions),Health Consumption Expenditures,Personal Health Care,Government Administration and Net Cost of Health Insurance,Government Public Health Activities,Investment,U.S. Population1 (Millions),Gross Domestic Product2 (Amount in Billions),National Health Expenditures (Annual Percent Change),Health Consumption Expenditures,Personal Health Care,Government Administration and Net Cost of Health Insurance,Government Public Health Activities,Investment,U.S. Population1,Gross Domestic Product2,National Health Expenditures (Percent Distribution),Health Consumption Expenditures,Personal Health Care,Government Administration and Net Cost of Health Insurance,Government Public Health Activities,Investment,National Health Expenditures (Per Capita Amount),Health Consumption Expenditures,Personal Health Care,Government Administration and Net Cost of Health Insurance,Government Public Health Activities,Investment,National Health Expenditures as a Percent of Gross Domestic Product (Percent),"Life expectancy at birth, total (years)"
0,1980,253.2,232.7,214.3,11.9,6.4,20.6,230.4,2857.3,15.3,15.1,15.9,0,19.9,17.5,1,8.8,100,91.9,84.6,4.7,2.5,8.1,1099,1010,930,52,28,89,8.9,73.609756
1,1981,293.6,270.1,248.6,13.9,7.5,23.5,232.7,3207,15.9,16.1,16,16.9,16.9,14.3,1,12.2,100,92,84.7,4.7,2.6,8,1262,1161,1068,60,32,101,9.2,74.009756
2,1982,330.9,304.2,279.5,16.1,8.6,26.7,235,3343.8,12.7,12.6,12.4,15.8,14.5,13.6,1,4.3,100,91.9,84.5,4.9,2.6,8.1,1408,1295,1189,69,37,114,9.9,74.360976
3,1983,364.8,335.5,308,18.3,9.3,29.3,237.2,3634,10.2,10.3,10.2,13.3,7.3,9.7,1,8.7,100,92,84.4,5,2.5,8,1538,1414,1298,77,39,123,10,74.463415
4,1984,401.9,371,337.9,23.3,9.8,30.9,239.4,4037.6,10.2,10.6,9.7,27.3,6.4,5.6,0.9,11.1,100,92.3,84.1,5.8,2.4,7.7,1679,1550,1412,97,41,129,10,74.563415


In [10]:
datum.shape

(37, 31)

### 3. Childhood Mortality Rates

In [11]:
child_mort = pd.read_csv('Childhood_Mortality_Over_Years.csv')
child_mort = child_mort[child_mort['Entity'] == 'United States'].reset_index(drop=True) # filter by U.S as well
child_mort = child_mort.drop(columns=['Entity', 'Code'])
child_mort = child_mort[child_mort['Year'] >= 1980].reset_index(drop=True)

child_mort.head()

,Year,Youth mortality rate (UN IGME (2018))
0,1980,1.795500
1,1981,1.725824
2,1982,1.666108
3,1983,1.606382
4,1984,1.566490


In [12]:
datum = pd.merge(datum, child_mort, on='Year')

datum.head() # earliest year should still be 1980

,Year,National Health Expenditures (Amount in Billions),Health Consumption Expenditures,Personal Health Care,Government Administration and Net Cost of Health Insurance,Government Public Health Activities,Investment,U.S. Population1 (Millions),Gross Domestic Product2 (Amount in Billions),National Health Expenditures (Annual Percent Change),Health Consumption Expenditures,Personal Health Care,Government Administration and Net Cost of Health Insurance,Government Public Health Activities,Investment,U.S. Population1,Gross Domestic Product2,National Health Expenditures (Percent Distribution),Health Consumption Expenditures,Personal Health Care,Government Administration and Net Cost of Health Insurance,Government Public Health Activities,Investment,National Health Expenditures (Per Capita Amount),Health Consumption Expenditures,Personal Health Care,Government Administration and Net Cost of Health Insurance,Government Public Health Activities,Investment,National Health Expenditures as a Percent of Gross Domestic Product (Percent),"Life expectancy at birth, total (years)",Youth mortality rate (UN IGME (2018))
0,1980,253.2,232.7,214.3,11.9,6.4,20.6,230.4,2857.3,15.3,15.1,15.9,0,19.9,17.5,1,8.8,100,91.9,84.6,4.7,2.5,8.1,1099,1010,930,52,28,89,8.9,73.609756,1.795500
1,1981,293.6,270.1,248.6,13.9,7.5,23.5,232.7,3207,15.9,16.1,16,16.9,16.9,14.3,1,12.2,100,92,84.7,4.7,2.6,8,1262,1161,1068,60,32,101,9.2,74.009756,1.725824
2,1982,330.9,304.2,279.5,16.1,8.6,26.7,235,3343.8,12.7,12.6,12.4,15.8,14.5,13.6,1,4.3,100,91.9,84.5,4.9,2.6,8.1,1408,1295,1189,69,37,114,9.9,74.360976,1.666108
3,1983,364.8,335.5,308,18.3,9.3,29.3,237.2,3634,10.2,10.3,10.2,13.3,7.3,9.7,1,8.7,100,92,84.4,5,2.5,8,1538,1414,1298,77,39,123,10,74.463415,1.606382
4,1984,401.9,371,337.9,23.3,9.8,30.9,239.4,4037.6,10.2,10.6,9.7,27.3,6.4,5.6,0.9,11.1,100,92.3,84.1,5.8,2.4,7.7,1679,1550,1412,97,41,129,10,74.563415,1.566490


In [13]:
datum.shape[0] == 37 # checking year frame is preserved...

True

### 4. Annual Number of Deaths

In [14]:
annual_deaths = pd.read_csv('annual-number-of-deaths-by-world-region.csv')
annual_deaths.head()

,Entity,Code,Year,"Estimates, 1950 - 2020: Annually interpolated demographic indicators - Deaths (thousands)"
0,Afghanistan,AFG,1950,298139.0
1,Afghanistan,AFG,1951,297342.0
2,Afghanistan,AFG,1952,295841.0
3,Afghanistan,AFG,1953,294524.0
4,Afghanistan,AFG,1954,293392.0


In [15]:
annual_deaths = annual_deaths[annual_deaths['Entity'] == 'United States']
annual_deaths = annual_deaths.drop(columns=['Entity', 'Code'])
annual_deaths = annual_deaths[annual_deaths['Year'] >= 1980].reset_index(drop=True)
annual_deaths = annual_deaths.reset_index(drop=True)

annual_deaths.head()

,Year,"Estimates, 1950 - 2020: Annually interpolated demographic indicators - Deaths (thousands)"
0,1980,2017882.0
1,1981,2037675.0
2,1982,2061284.0
3,1983,2087053.0
4,1984,2113891.0


In [16]:
#merge annual deaths data on year
datum = pd.merge(datum, annual_deaths, on='Year')

datum.head() # earliest year should still be 1980

,Year,National Health Expenditures (Amount in Billions),Health Consumption Expenditures,Personal Health Care,Government Administration and Net Cost of Health Insurance,Government Public Health Activities,Investment,U.S. Population1 (Millions),Gross Domestic Product2 (Amount in Billions),National Health Expenditures (Annual Percent Change),Health Consumption Expenditures,Personal Health Care,Government Administration and Net Cost of Health Insurance,Government Public Health Activities,Investment,U.S. Population1,Gross Domestic Product2,National Health Expenditures (Percent Distribution),Health Consumption Expenditures,Personal Health Care,Government Administration and Net Cost of Health Insurance,Government Public Health Activities,Investment,National Health Expenditures (Per Capita Amount),Health Consumption Expenditures,Personal Health Care,Government Administration and Net Cost of Health Insurance,Government Public Health Activities,Investment,National Health Expenditures as a Percent of Gross Domestic Product (Percent),"Life expectancy at birth, total (years)",Youth mortality rate (UN IGME (2018)),"Estimates, 1950 - 2020: Annually interpolated demographic indicators - Deaths (thousands)"
0,1980,253.2,232.7,214.3,11.9,6.4,20.6,230.4,2857.3,15.3,15.1,15.9,0,19.9,17.5,1,8.8,100,91.9,84.6,4.7,2.5,8.1,1099,1010,930,52,28,89,8.9,73.609756,1.795500,2017882.0
1,1981,293.6,270.1,248.6,13.9,7.5,23.5,232.7,3207,15.9,16.1,16,16.9,16.9,14.3,1,12.2,100,92,84.7,4.7,2.6,8,1262,1161,1068,60,32,101,9.2,74.009756,1.725824,2037675.0
2,1982,330.9,304.2,279.5,16.1,8.6,26.7,235,3343.8,12.7,12.6,12.4,15.8,14.5,13.6,1,4.3,100,91.9,84.5,4.9,2.6,8.1,1408,1295,1189,69,37,114,9.9,74.360976,1.666108,2061284.0
3,1983,364.8,335.5,308,18.3,9.3,29.3,237.2,3634,10.2,10.3,10.2,13.3,7.3,9.7,1,8.7,100,92,84.4,5,2.5,8,1538,1414,1298,77,39,123,10,74.463415,1.606382,2087053.0
4,1984,401.9,371,337.9,23.3,9.8,30.9,239.4,4037.6,10.2,10.6,9.7,27.3,6.4,5.6,0.9,11.1,100,92.3,84.1,5.8,2.4,7.7,1679,1550,1412,97,41,129,10,74.563415,1.566490,2113891.0


In [17]:
datum.shape[0] == 37 # checking year frame is preserved...

True

###5. Vaccination Rate Over Years

In [18]:
vaccination_rate = pd.read_csv('vaccination-coverage-by-income-in.csv')
vaccination_rate.head()

,Entity,Code,Year,DTP3 (% of one-year-olds immunized),Output-side real GDP per capita (gdppc_o) (PWT 9.1 (2019)),"Total population (Gapminder, HYDE & UN)",Continent
0,Abkhazia,OWID_ABK,2015,NaN,NaN,NaN,Asia
1,Afghanistan,AFG,1980,4.0,NaN,13357000.0,NaN
2,Afghanistan,AFG,1981,3.0,NaN,13172000.0,NaN
3,Afghanistan,AFG,1982,5.0,NaN,12883000.0,NaN
4,Afghanistan,AFG,1983,5.0,NaN,12538000.0,NaN


In [19]:
# We only need the colummns containing info on the US and >= 1980
vaccination_rate = vaccination_rate[vaccination_rate['Entity'] == 'United States']
vaccination_rate = vaccination_rate.drop(columns=['Entity', 'Code'])
vaccination_rate = vaccination_rate[vaccination_rate['Year'] >= 1980].reset_index(drop=True)
vaccination_rate = vaccination_rate.reset_index(drop=True)

vaccination_rate.head()

,Year,DTP3 (% of one-year-olds immunized),Output-side real GDP per capita (gdppc_o) (PWT 9.1 (2019)),"Total population (Gapminder, HYDE & UN)",Continent
0,1980,96.0,29154.891,229476000.0,NaN
1,1981,96.0,29631.477,231636000.0,NaN
2,1982,96.0,28651.811,233822000.0,NaN
3,1983,97.0,29674.672,236030000.0,NaN
4,1984,97.0,31669.826,238256992.0,NaN


In [20]:
#merge vaccination data on year
datum = pd.merge(datum, vaccination_rate, on='Year')

datum.head() # earliest year should still be 1980

,Year,National Health Expenditures (Amount in Billions),Health Consumption Expenditures,Personal Health Care,Government Administration and Net Cost of Health Insurance,Government Public Health Activities,Investment,U.S. Population1 (Millions),Gross Domestic Product2 (Amount in Billions),National Health Expenditures (Annual Percent Change),Health Consumption Expenditures,Personal Health Care,Government Administration and Net Cost of Health Insurance,Government Public Health Activities,Investment,U.S. Population1,Gross Domestic Product2,National Health Expenditures (Percent Distribution),Health Consumption Expenditures,Personal Health Care,Government Administration and Net Cost of Health Insurance,Government Public Health Activities,Investment,National Health Expenditures (Per Capita Amount),Health Consumption Expenditures,Personal Health Care,Government Administration and Net Cost of Health Insurance,Government Public Health Activities,Investment,National Health Expenditures as a Percent of Gross Domestic Product (Percent),"Life expectancy at birth, total (years)",Youth mortality rate (UN IGME (2018)),"Estimates, 1950 - 2020: Annually interpolated demographic indicators - Deaths (thousands)",DTP3 (% of one-year-olds immunized),Output-side real GDP per capita (gdppc_o) (PWT 9.1 (2019)),"Total population (Gapminder, HYDE & UN)",Continent
0,1980,253.2,232.7,214.3,11.9,6.4,20.6,230.4,2857.3,15.3,15.1,15.9,0,19.9,17.5,1,8.8,100,91.9,84.6,4.7,2.5,8.1,1099,1010,930,52,28,89,8.9,73.609756,1.795500,2017882.0,96.0,29154.891,229476000.0,NaN
1,1981,293.6,270.1,248.6,13.9,7.5,23.5,232.7,3207,15.9,16.1,16,16.9,16.9,14.3,1,12.2,100,92,84.7,4.7,2.6,8,1262,1161,1068,60,32,101,9.2,74.009756,1.725824,2037675.0,96.0,29631.477,231636000.0,NaN
2,1982,330.9,304.2,279.5,16.1,8.6,26.7,235,3343.8,12.7,12.6,12.4,15.8,14.5,13.6,1,4.3,100,91.9,84.5,4.9,2.6,8.1,1408,1295,1189,69,37,114,9.9,74.360976,1.666108,2061284.0,96.0,28651.811,233822000.0,NaN
3,1983,364.8,335.5,308,18.3,9.3,29.3,237.2,3634,10.2,10.3,10.2,13.3,7.3,9.7,1,8.7,100,92,84.4,5,2.5,8,1538,1414,1298,77,39,123,10,74.463415,1.606382,2087053.0,97.0,29674.672,236030000.0,NaN
4,1984,401.9,371,337.9,23.3,9.8,30.9,239.4,4037.6,10.2,10.6,9.7,27.3,6.4,5.6,0.9,11.1,100,92.3,84.1,5.8,2.4,7.7,1679,1550,1412,97,41,129,10,74.563415,1.566490,2113891.0,97.0,31669.826,238256992.0,NaN


In [21]:
datum.shape[0] == 37 # checking year frame is preserved...

True

### 6. Insurance Rate Over the Years

In [22]:
insurance_rate = pd.read_csv('Insurance_Rate_Over_Years.csv')
insurance_rate.head()

,Entity,Code,Year,Percentage of persons without health insurance (%)
0,United States,USA,1963,24.3
1,United States,USA,1968,14.5
2,United States,USA,1970,14.6
3,United States,USA,1972,13.2
4,United States,USA,1974,11.4


In [23]:
insurance_rate = insurance_rate[insurance_rate['Entity'] == 'United States']
insurance_rate = insurance_rate.drop(columns=['Entity', 'Code'])
insurance_rate = insurance_rate[insurance_rate['Year'] >= 1980].reset_index(drop=True)
insurance_rate = insurance_rate.reset_index(drop=True)

insurance_rate.head()

,Year,Percentage of persons without health insurance (%)
0,1980,11.4
1,1982,12.5
2,1983,13.1
3,1984,12.9
4,1986,13.3


In [24]:
datum = datum.set_index('Year')
insurance_rate = insurance_rate.set_index('Year')
datum['Percentage of persons without health insurance (%)'] = insurance_rate['Percentage of persons without health insurance (%)']
datum.head()

,National Health Expenditures (Amount in Billions),Health Consumption Expenditures,Personal Health Care,Government Administration and Net Cost of Health Insurance,Government Public Health Activities,Investment,U.S. Population1 (Millions),Gross Domestic Product2 (Amount in Billions),National Health Expenditures (Annual Percent Change),Health Consumption Expenditures,Personal Health Care,Government Administration and Net Cost of Health Insurance,Government Public Health Activities,Investment,U.S. Population1,Gross Domestic Product2,National Health Expenditures (Percent Distribution),Health Consumption Expenditures,Personal Health Care,Government Administration and Net Cost of Health Insurance,Government Public Health Activities,Investment,National Health Expenditures (Per Capita Amount),Health Consumption Expenditures,Personal Health Care,Government Administration and Net Cost of Health Insurance,Government Public Health Activities,Investment,National Health Expenditures as a Percent of Gross Domestic Product (Percent),"Life expectancy at birth, total (years)",Youth mortality rate (UN IGME (2018)),"Estimates, 1950 - 2020: Annually interpolated demographic indicators - Deaths (thousands)",DTP3 (% of one-year-olds immunized),Output-side real GDP per capita (gdppc_o) (PWT 9.1 (2019)),"Total population (Gapminder, HYDE & UN)",Continent,Percentage of persons without health insurance (%)
Year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1980.0,253.2,232.7,214.3,11.9,6.4,20.6,230.4,2857.3,15.3,15.1,15.9,0,19.9,17.5,1,8.8,100,91.9,84.6,4.7,2.5,8.1,1099,1010,930,52,28,89,8.9,73.609756,1.795500,2017882.0,96.0,29154.891,229476000.0,NaN,11.4
1981.0,293.6,270.1,248.6,13.9,7.5,23.5,232.7,3207,15.9,16.1,16,16.9,16.9,14.3,1,12.2,100,92,84.7,4.7,2.6,8,1262,1161,1068,60,32,101,9.2,74.009756,1.725824,2037675.0,96.0,29631.477,231636000.0,NaN,NaN
1982.0,330.9,304.2,279.5,16.1,8.6,26.7,235,3343.8,12.7,12.6,12.4,15.8,14.5,13.6,1,4.3,100,91.9,84.5,4.9,2.6,8.1,1408,1295,1189,69,37,114,9.9,74.360976,1.666108,2061284.0,96.0,28651.811,233822000.0,NaN,12.5
1983.0,364.8,335.5,308,18.3,9.3,29.3,237.2,3634,10.2,10.3,10.2,13.3,7.3,9.7,1,8.7,100,92,84.4,5,2.5,8,1538,1414,1298,77,39,123,10,74.463415,1.606382,2087053.0,97.0,29674.672,236030000.0,NaN,13.1
1984.0,401.9,371,337.9,23.3,9.8,30.9,239.4,4037.6,10.2,10.6,9.7,27.3,6.4,5.6,0.9,11.1,100,92.3,84.1,5.8,2.4,7.7,1679,1550,1412,97,41,129,10,74.563415,1.566490,2113891.0,97.0,31669.826,238256992.0,NaN,12.9


In [25]:
datum = datum.reset_index()
datum.head()

,Year,National Health Expenditures (Amount in Billions),Health Consumption Expenditures,Personal Health Care,Government Administration and Net Cost of Health Insurance,Government Public Health Activities,Investment,U.S. Population1 (Millions),Gross Domestic Product2 (Amount in Billions),National Health Expenditures (Annual Percent Change),Health Consumption Expenditures,Personal Health Care,Government Administration and Net Cost of Health Insurance,Government Public Health Activities,Investment,U.S. Population1,Gross Domestic Product2,National Health Expenditures (Percent Distribution),Health Consumption Expenditures,Personal Health Care,Government Administration and Net Cost of Health Insurance,Government Public Health Activities,Investment,National Health Expenditures (Per Capita Amount),Health Consumption Expenditures,Personal Health Care,Government Administration and Net Cost of Health Insurance,Government Public Health Activities,Investment,National Health Expenditures as a Percent of Gross Domestic Product (Percent),"Life expectancy at birth, total (years)",Youth mortality rate (UN IGME (2018)),"Estimates, 1950 - 2020: Annually interpolated demographic indicators - Deaths (thousands)",DTP3 (% of one-year-olds immunized),Output-side real GDP per capita (gdppc_o) (PWT 9.1 (2019)),"Total population (Gapminder, HYDE & UN)",Continent,Percentage of persons without health insurance (%)
0,1980.0,253.2,232.7,214.3,11.9,6.4,20.6,230.4,2857.3,15.3,15.1,15.9,0,19.9,17.5,1,8.8,100,91.9,84.6,4.7,2.5,8.1,1099,1010,930,52,28,89,8.9,73.609756,1.795500,2017882.0,96.0,29154.891,229476000.0,NaN,11.4
1,1981.0,293.6,270.1,248.6,13.9,7.5,23.5,232.7,3207,15.9,16.1,16,16.9,16.9,14.3,1,12.2,100,92,84.7,4.7,2.6,8,1262,1161,1068,60,32,101,9.2,74.009756,1.725824,2037675.0,96.0,29631.477,231636000.0,NaN,NaN
2,1982.0,330.9,304.2,279.5,16.1,8.6,26.7,235,3343.8,12.7,12.6,12.4,15.8,14.5,13.6,1,4.3,100,91.9,84.5,4.9,2.6,8.1,1408,1295,1189,69,37,114,9.9,74.360976,1.666108,2061284.0,96.0,28651.811,233822000.0,NaN,12.5
3,1983.0,364.8,335.5,308,18.3,9.3,29.3,237.2,3634,10.2,10.3,10.2,13.3,7.3,9.7,1,8.7,100,92,84.4,5,2.5,8,1538,1414,1298,77,39,123,10,74.463415,1.606382,2087053.0,97.0,29674.672,236030000.0,NaN,13.1
4,1984.0,401.9,371,337.9,23.3,9.8,30.9,239.4,4037.6,10.2,10.6,9.7,27.3,6.4,5.6,0.9,11.1,100,92.3,84.1,5.8,2.4,7.7,1679,1550,1412,97,41,129,10,74.563415,1.566490,2113891.0,97.0,31669.826,238256992.0,NaN,12.9


In [26]:
datum = datum.drop(columns = ['Continent'])
datum.head()


,Year,National Health Expenditures (Amount in Billions),Health Consumption Expenditures,Personal Health Care,Government Administration and Net Cost of Health Insurance,Government Public Health Activities,Investment,U.S. Population1 (Millions),Gross Domestic Product2 (Amount in Billions),National Health Expenditures (Annual Percent Change),Health Consumption Expenditures,Personal Health Care,Government Administration and Net Cost of Health Insurance,Government Public Health Activities,Investment,U.S. Population1,Gross Domestic Product2,National Health Expenditures (Percent Distribution),Health Consumption Expenditures,Personal Health Care,Government Administration and Net Cost of Health Insurance,Government Public Health Activities,Investment,National Health Expenditures (Per Capita Amount),Health Consumption Expenditures,Personal Health Care,Government Administration and Net Cost of Health Insurance,Government Public Health Activities,Investment,National Health Expenditures as a Percent of Gross Domestic Product (Percent),"Life expectancy at birth, total (years)",Youth mortality rate (UN IGME (2018)),"Estimates, 1950 - 2020: Annually interpolated demographic indicators - Deaths (thousands)",DTP3 (% of one-year-olds immunized),Output-side real GDP per capita (gdppc_o) (PWT 9.1 (2019)),"Total population (Gapminder, HYDE & UN)",Percentage of persons without health insurance (%)
0,1980.0,253.2,232.7,214.3,11.9,6.4,20.6,230.4,2857.3,15.3,15.1,15.9,0,19.9,17.5,1,8.8,100,91.9,84.6,4.7,2.5,8.1,1099,1010,930,52,28,89,8.9,73.609756,1.795500,2017882.0,96.0,29154.891,229476000.0,11.4
1,1981.0,293.6,270.1,248.6,13.9,7.5,23.5,232.7,3207,15.9,16.1,16,16.9,16.9,14.3,1,12.2,100,92,84.7,4.7,2.6,8,1262,1161,1068,60,32,101,9.2,74.009756,1.725824,2037675.0,96.0,29631.477,231636000.0,NaN
2,1982.0,330.9,304.2,279.5,16.1,8.6,26.7,235,3343.8,12.7,12.6,12.4,15.8,14.5,13.6,1,4.3,100,91.9,84.5,4.9,2.6,8.1,1408,1295,1189,69,37,114,9.9,74.360976,1.666108,2061284.0,96.0,28651.811,233822000.0,12.5
3,1983.0,364.8,335.5,308,18.3,9.3,29.3,237.2,3634,10.2,10.3,10.2,13.3,7.3,9.7,1,8.7,100,92,84.4,5,2.5,8,1538,1414,1298,77,39,123,10,74.463415,1.606382,2087053.0,97.0,29674.672,236030000.0,13.1
4,1984.0,401.9,371,337.9,23.3,9.8,30.9,239.4,4037.6,10.2,10.6,9.7,27.3,6.4,5.6,0.9,11.1,100,92.3,84.1,5.8,2.4,7.7,1679,1550,1412,97,41,129,10,74.563415,1.566490,2113891.0,97.0,31669.826,238256992.0,12.9


## Final Cleaning  & Final Dataset

In [27]:
datum.head()

,Year,National Health Expenditures (Amount in Billions),Health Consumption Expenditures,Personal Health Care,Government Administration and Net Cost of Health Insurance,Government Public Health Activities,Investment,U.S. Population1 (Millions),Gross Domestic Product2 (Amount in Billions),National Health Expenditures (Annual Percent Change),Health Consumption Expenditures,Personal Health Care,Government Administration and Net Cost of Health Insurance,Government Public Health Activities,Investment,U.S. Population1,Gross Domestic Product2,National Health Expenditures (Percent Distribution),Health Consumption Expenditures,Personal Health Care,Government Administration and Net Cost of Health Insurance,Government Public Health Activities,Investment,National Health Expenditures (Per Capita Amount),Health Consumption Expenditures,Personal Health Care,Government Administration and Net Cost of Health Insurance,Government Public Health Activities,Investment,National Health Expenditures as a Percent of Gross Domestic Product (Percent),"Life expectancy at birth, total (years)",Youth mortality rate (UN IGME (2018)),"Estimates, 1950 - 2020: Annually interpolated demographic indicators - Deaths (thousands)",DTP3 (% of one-year-olds immunized),Output-side real GDP per capita (gdppc_o) (PWT 9.1 (2019)),"Total population (Gapminder, HYDE & UN)",Percentage of persons without health insurance (%)
0,1980.0,253.2,232.7,214.3,11.9,6.4,20.6,230.4,2857.3,15.3,15.1,15.9,0,19.9,17.5,1,8.8,100,91.9,84.6,4.7,2.5,8.1,1099,1010,930,52,28,89,8.9,73.609756,1.795500,2017882.0,96.0,29154.891,229476000.0,11.4
1,1981.0,293.6,270.1,248.6,13.9,7.5,23.5,232.7,3207,15.9,16.1,16,16.9,16.9,14.3,1,12.2,100,92,84.7,4.7,2.6,8,1262,1161,1068,60,32,101,9.2,74.009756,1.725824,2037675.0,96.0,29631.477,231636000.0,NaN
2,1982.0,330.9,304.2,279.5,16.1,8.6,26.7,235,3343.8,12.7,12.6,12.4,15.8,14.5,13.6,1,4.3,100,91.9,84.5,4.9,2.6,8.1,1408,1295,1189,69,37,114,9.9,74.360976,1.666108,2061284.0,96.0,28651.811,233822000.0,12.5
3,1983.0,364.8,335.5,308,18.3,9.3,29.3,237.2,3634,10.2,10.3,10.2,13.3,7.3,9.7,1,8.7,100,92,84.4,5,2.5,8,1538,1414,1298,77,39,123,10,74.463415,1.606382,2087053.0,97.0,29674.672,236030000.0,13.1
4,1984.0,401.9,371,337.9,23.3,9.8,30.9,239.4,4037.6,10.2,10.6,9.7,27.3,6.4,5.6,0.9,11.1,100,92.3,84.1,5.8,2.4,7.7,1679,1550,1412,97,41,129,10,74.563415,1.566490,2113891.0,97.0,31669.826,238256992.0,12.9


In [28]:
datum_columns = ['Year', 'Health_Spending', 'Health_Consumption_Spending', 'Personal_Health', 'Administration_Cost', 'Public_Health_Activities',
                 'Investment', 'Population', 'GDP', 'Annual_Pct_Change_Health_Spending', 'Annual_Pct_Change_Health_Consumption', 'Annual_Pct_Change_Personal_Health', 
                 'Annual_Pct_Change_Administration_Cost', 'Annual_Pct_Change_Public_Health_Activities', 
                 'Annual_Pct_Change_Investment', 'Annual_Pct_Change_Population', 'Annual_Pct_Change_GDP', 'Percent_Spent_National_Health',
                 'Percent_Spent_Health_Consumption', 'Percent_Spent_Personal_Health', 'Percent_Spent_Administration_Cost', 'Percent_Spent_Public_Health_Activities',
                 'Percent_Spent_Investment', 'Per_Capita_National_Health', 'Per_Capita_Health_Consumption', 'Per_Capita_Personal_Health', 'Per_Capita_Administration_Cost', 
                 'Per_Capita_Public_Health_Activities', 'Per_Capita_Investment', 'Pct_GDP_on_National_Health', 'Life_Expectancy', 'Youth_Mortality_Rate', 'Annual_Deaths', 
                 'Vaccination_Rate', 'Per_Capita_Real_GDP', 'Total_Population', 'Pct_Uninsured_Rate']

In [29]:
datum.columns = datum_columns

In [30]:
datum = datum.drop(columns=['Total_Population'])

In [31]:
datum.head()

,Year,Health_Spending,Health_Consumption_Spending,Personal_Health,Administration_Cost,Public_Health_Activities,Investment,Population,GDP,Annual_Pct_Change_Health_Spending,Annual_Pct_Change_Health_Consumption,Annual_Pct_Change_Personal_Health,Annual_Pct_Change_Administration_Cost,Annual_Pct_Change_Public_Health_Activities,Annual_Pct_Change_Investment,Annual_Pct_Change_Population,Annual_Pct_Change_GDP,Percent_Spent_National_Health,Percent_Spent_Health_Consumption,Percent_Spent_Personal_Health,Percent_Spent_Administration_Cost,Percent_Spent_Public_Health_Activities,Percent_Spent_Investment,Per_Capita_National_Health,Per_Capita_Health_Consumption,Per_Capita_Personal_Health,Per_Capita_Administration_Cost,Per_Capita_Public_Health_Activities,Per_Capita_Investment,Pct_GDP_on_National_Health,Life_Expectancy,Youth_Mortality_Rate,Annual_Deaths,Vaccination_Rate,Per_Capita_Real_GDP,Pct_Uninsured_Rate
0,1980.0,253.2,232.7,214.3,11.9,6.4,20.6,230.4,2857.3,15.3,15.1,15.9,0,19.9,17.5,1,8.8,100,91.9,84.6,4.7,2.5,8.1,1099,1010,930,52,28,89,8.9,73.609756,1.795500,2017882.0,96.0,29154.891,11.4
1,1981.0,293.6,270.1,248.6,13.9,7.5,23.5,232.7,3207,15.9,16.1,16,16.9,16.9,14.3,1,12.2,100,92,84.7,4.7,2.6,8,1262,1161,1068,60,32,101,9.2,74.009756,1.725824,2037675.0,96.0,29631.477,NaN
2,1982.0,330.9,304.2,279.5,16.1,8.6,26.7,235,3343.8,12.7,12.6,12.4,15.8,14.5,13.6,1,4.3,100,91.9,84.5,4.9,2.6,8.1,1408,1295,1189,69,37,114,9.9,74.360976,1.666108,2061284.0,96.0,28651.811,12.5
3,1983.0,364.8,335.5,308,18.3,9.3,29.3,237.2,3634,10.2,10.3,10.2,13.3,7.3,9.7,1,8.7,100,92,84.4,5,2.5,8,1538,1414,1298,77,39,123,10,74.463415,1.606382,2087053.0,97.0,29674.672,13.1
4,1984.0,401.9,371,337.9,23.3,9.8,30.9,239.4,4037.6,10.2,10.6,9.7,27.3,6.4,5.6,0.9,11.1,100,92.3,84.1,5.8,2.4,7.7,1679,1550,1412,97,41,129,10,74.563415,1.566490,2113891.0,97.0,31669.826,12.9


In [33]:
datum[['Health_Spending', 'Health_Consumption_Spending', 'Personal_Health', 'Administration_Cost', 'Public_Health_Activities', 'Investment', 'GDP']] = datum[['Health_Spending', 'Health_Consumption_Spending', 'Personal_Health', 'Administration_Cost', 'Public_Health_Activities', 'Investment', 'GDP']].apply(lambda x: x*pow(10, 9))

In [34]:
datum[['Population']] = datum[['Population']].apply(lambda x: x*pow(10, 6))

In [35]:
datum.head()

,Year,Health_Spending,Health_Consumption_Spending,Personal_Health,Administration_Cost,Public_Health_Activities,Investment,Population,GDP,Annual_Pct_Change_Health_Spending,Annual_Pct_Change_Health_Consumption,Annual_Pct_Change_Personal_Health,Annual_Pct_Change_Administration_Cost,Annual_Pct_Change_Public_Health_Activities,Annual_Pct_Change_Investment,Annual_Pct_Change_Population,Annual_Pct_Change_GDP,Percent_Spent_National_Health,Percent_Spent_Health_Consumption,Percent_Spent_Personal_Health,Percent_Spent_Administration_Cost,Percent_Spent_Public_Health_Activities,Percent_Spent_Investment,Per_Capita_National_Health,Per_Capita_Health_Consumption,Per_Capita_Personal_Health,Per_Capita_Administration_Cost,Per_Capita_Public_Health_Activities,Per_Capita_Investment,Pct_GDP_on_National_Health,Life_Expectancy,Youth_Mortality_Rate,Annual_Deaths,Vaccination_Rate,Per_Capita_Real_GDP,Pct_Uninsured_Rate
0,1980.0,2.532e+11,2.327e+11,2.143e+11,1.19e+10,6.4e+09,2.06e+10,2.304e+08,2.8573e+12,15.3,15.1,15.9,0,19.9,17.5,1,8.8,100,91.9,84.6,4.7,2.5,8.1,1099,1010,930,52,28,89,8.9,73.609756,1.795500,2017882.0,96.0,29154.891,11.4
1,1981.0,2.936e+11,2.701e+11,2.486e+11,1.39e+10,7.5e+09,2.35e+10,2.327e+08,3.207e+12,15.9,16.1,16,16.9,16.9,14.3,1,12.2,100,92,84.7,4.7,2.6,8,1262,1161,1068,60,32,101,9.2,74.009756,1.725824,2037675.0,96.0,29631.477,NaN
2,1982.0,3.309e+11,3.042e+11,2.795e+11,1.61e+10,8.6e+09,2.67e+10,2.35e+08,3.3438e+12,12.7,12.6,12.4,15.8,14.5,13.6,1,4.3,100,91.9,84.5,4.9,2.6,8.1,1408,1295,1189,69,37,114,9.9,74.360976,1.666108,2061284.0,96.0,28651.811,12.5
3,1983.0,3.648e+11,3.355e+11,3.08e+11,1.83e+10,9.3e+09,2.93e+10,2.372e+08,3.634e+12,10.2,10.3,10.2,13.3,7.3,9.7,1,8.7,100,92,84.4,5,2.5,8,1538,1414,1298,77,39,123,10,74.463415,1.606382,2087053.0,97.0,29674.672,13.1
4,1984.0,4.019e+11,3.71e+11,3.379e+11,2.33e+10,9.8e+09,3.09e+10,2.394e+08,4.0376e+12,10.2,10.6,9.7,27.3,6.4,5.6,0.9,11.1,100,92.3,84.1,5.8,2.4,7.7,1679,1550,1412,97,41,129,10,74.563415,1.566490,2113891.0,97.0,31669.826,12.9
